In [25]:
import os
import numpy as np
import torch
import torchvision
from torch import nn
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms, datasets
from torchvision.datasets import MNIST
from torchvision.utils import save_image
import matplotlib.pyplot as plt
import pylab
import torch.optim as optim
from torch.nn import functional as F

In [56]:
num_epochs = 100
batch_size = 128
seed = 1
out_dir = './vae_20'

In [57]:

cuda = torch.cuda.is_available()
if cuda:
    print('cuda is available!')

if not os.path.exists(out_dir):
    os.mkdir(out_dir)

torch.manual_seed(seed)
if cuda:
    torch.cuda.manual_seed(seed)

cuda is available!


In [58]:
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data',
                   train=True,
                   download=True,
                   transform=transforms.ToTensor()),
    batch_size=batch_size,
    shuffle=True
)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data',
                   train=False,
                   transform=transforms.ToTensor()),
    batch_size=batch_size,
    shuffle=True
)


In [59]:
print(train_loader)

In [60]:

class VAE(nn.Module):

    def __init__(self):
        super(VAE, self).__init__()
        
        self.fc1 = nn.Linear(784, 512)
        self.fc21 = nn.Linear(512, 20)  # mu
        self.fc22 = nn.Linear(512, 20)  # logvar

        self.fc3 = nn.Linear(20, 512)
        self.fc4 = nn.Linear(512, 784)
        
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
    
    def encode(self, x):
        h = self.relu(self.fc1(x))
        return self.fc21(h), self.fc22(h)

    def reparameterize(self, mu, logvar):
        if self.training:
            std = logvar.mul(0.5).exp_()
            eps = Variable(std.data.new(std.size()).normal_())
            return eps.mul(std).add_(mu)
        else:
            return mu

    def decode(self, z):
        h = self.relu(self.fc3(z))
        return self.sigmoid(self.fc4(h))
    
    def forward(self, x):
        x = x.view(-1, 784)
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

model = VAE()
if cuda:
    model.cuda()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [61]:

def loss_function(recon_x, x, mu, logvar):
    # size_average=Falseなのでバッチ内のサンプルの合計lossを求める
    # reconstruction loss 入力画像をどのくらい正確に復元できたか？
    # 数式では対数尤度の最大化だが交差エントロピーlossの最小化と等価
    recon = F.binary_cross_entropy(recon_x, x.view(-1, 784), size_average=False)

    # 潜在空間zに対する正則化項
    # P(z|x) が N(0, I)に近くなる（KL-distanceが小さくなる）ようにする
    kld = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return recon + kld

In [62]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        if cuda:
            data = Variable(data.cuda())
        else:
            data = Variable(data)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
    
    # loss_function() は平均ではなく全サンプルの合計lossを返すのでサンプル数で割る
    train_loss /= len(train_loader.dataset)

    return train_loss    


In [63]:

def test(epoch):
    model.eval()
    test_loss = 0
    for batch_idx, (data, _) in enumerate(test_loader):
        if cuda:
            data = Variable(data.cuda(), volatile=True)
        else:
            data = Variable(data, volatile=True)
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        test_loss += loss.item()
        
        if epoch % 10 == 0:
            # 10エポックごとに最初のminibatchの入力画像と復元画像を保存
            if batch_idx == 0:
                n = 8
                comparison = torch.cat([data[:n],
                                        recon_batch.view(batch_size, 1, 28, 28)[:n]])
                save_image(comparison.data.cpu(),
                           '{}/reconstruction_{}_20.png'.format(out_dir, epoch), nrow=n)

    test_loss /= len(test_loader.dataset)

    return test_loss

In [64]:
loss_list = []
test_loss_list = []
for epoch in range(1, num_epochs + 1):
    loss = train(epoch)
    test_loss = test(epoch)

    print('epoch [{}/{}], loss: {:.4f} test_loss: {:.4f}'.format(
        epoch + 1,
        num_epochs,
        loss,
        test_loss))

    # logging
    loss_list.append(loss)
    test_loss_list.append(test_loss)

# save the training model
np.save('./vae_20/loss_list_20.npy', np.array(loss_list))
np.save('./vae_20/test_loss_list_20.npy', np.array(test_loss_list))
torch.save(model.state_dict(), './vae_20/vae_20.pth')


/home/user/PycharmProjects/auto_encoder/venv/lib/python3.6/site-packages/ipykernel_launcher.py:7: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  import sys


epoch [2/100], loss: 158.7105 test_loss: 115.2169
epoch [3/100], loss: 118.4853 test_loss: 104.9736
epoch [4/100], loss: 112.7418 test_loss: 101.1061
epoch [5/100], loss: 110.1834 test_loss: 100.1671
epoch [6/100], loss: 108.6221 test_loss: 99.0382
epoch [7/100], loss: 107.5751 test_loss: 98.0739
epoch [8/100], loss: 106.8707 test_loss: 97.2110
epoch [9/100], loss: 106.2958 test_loss: 96.7292
epoch [10/100], loss: 105.8637 test_loss: 96.1738
epoch [11/100], loss: 105.4890 test_loss: 95.9200
epoch [12/100], loss: 105.1402 test_loss: 96.2292
epoch [13/100], loss: 104.8197 test_loss: 95.6445
epoch [14/100], loss: 104.6068 test_loss: 95.8513
epoch [15/100], loss: 104.4030 test_loss: 95.6735
epoch [16/100], loss: 104.1800 test_loss: 94.9153
epoch [17/100], loss: 104.0063 test_loss: 95.3870
epoch [18/100], loss: 103.8504 test_loss: 95.4805
epoch [19/100], loss: 103.6045 test_loss: 95.2434
epoch [20/100], loss: 103.5206 test_loss: 95.3382
epoch [21/100], loss: 103.3319 test_loss: 95.2700
epoc

In [5]:

import matplotlib.pyplot as plt
%matplotlib inline
loss_list = np.load('{}/loss_list_20.npy'.format(out_dir))
plt.plot(loss_list)
plt.xlabel('epoch')
plt.ylabel('loss')
plt.grid()

NameError: name 'np' is not defined

In [6]:
test_loss_list = np.load('{}/test_loss_list_20.npy'.format(out_dir))
plt.plot(test_loss_list)
plt.xlabel('epoch')
plt.ylabel('test loss')
plt.grid()

NameError: name 'np' is not defined

In [7]:
model.load_state_dict(torch.load('{}/vae.pth'.format(out_dir),
                                 map_location=lambda storage,
                                 loc: storage))
test_dataset = datasets.MNIST('./data', download=True, train=False, transform=transforms.ToTensor())
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=10000, shuffle=False)
images, labels = iter(test_loader).next()
images = images.view(10000, -1)


NameError: name 'model' is not defined

In [50]:
# 784次元ベクトルを2次元ベクトルにencode　ｚが２次元の場合をプロットしている。２０の場合はT-SNEなどを使って前処理する必要がありそう。
z = model.encode(Variable(images, volatile=True).cuda())
mu, logvar = z
mu, logvar = mu.cpu().data.numpy(), logvar.cpu().data.numpy()
print(mu.shape, logvar.shape)

(10000, 2) (10000, 2)


/home/user/PycharmProjects/auto_encoder/venv/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  


In [4]:

# plt.scatterは分布を図示する関数。makerには星や丸などの形が定義されており、c,cmapで色を定義している。
plt.figure(figsize=(10, 10))
plt.scatter(mu[:, 0], mu[:, 1], marker='.', c=labels.numpy(), cmap=pylab.cm.jet)
plt.colorbar()
plt.xlim((-6, 6))
plt.ylim((-6, 6))
plt.grid()

NameError: name 'plt' is not defined